# NPS

The customer of this study is a large telecommunications company that provides services throughout the CIS. The company is faced with the task of determining the current level of consumer loyalty, or NPS, among customers from Russia.


To determine the level of loyalty, customers were asked the classic question: "On a scale of 1 to 10, rate the likelihood that you would recommend the company to friends and acquaintances."


The company conducted a survey and asked to prepare a dashboard with its results. They did not deploy a large database for such a task and uploaded the data to SQLite.


To evaluate the results of the survey, the assessments are usually divided into three groups:
- 9-10 points - `promoters` 
- 7-8 points - `passives` 
- 0-6 points - `detractors` 

In [2]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [3]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [4]:
query = """
SELECT * FROM location;
"""

In [5]:
df = pd.read_sql(query, engine)
df.head(3)

,location_id,city,country
0,1,Архангельск,Россия
1,2,Астрахань,Россия
2,3,Балашиха,Россия


In [6]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

In [11]:
query_1 = """
SELECT u.user_id,
u.lt_day,
    CASE 
        WHEN lt_day <= 365 THEN 'Новый'
        ELSE 'Старый'
    END AS is_new,
u.age,
    CASE
        WHEN gender_segment == 0 THEN 'мужчина'
        WHEN gender_segment == 1 THEN 'женщина'
        ELSE 'None'
    END AS gender_segment,
u.os_name,
u.cpe_type_name,
u.nps_score,
    CASE
        WHEN nps_score >=9 THEN 'cторонники'
        WHEN nps_score >=7 AND nps_score <=8 THEN 'нейтралы'
        WHEN nps_score <=6 THEN 'критики'
    END AS nps_group,
l.country,
l.city,
a.title AS age_segment,
t.title AS traffic_segment,
lt.title AS lifetime_segment
FROM user AS u
LEFT JOIN location AS l ON u.location_id = l.location_id
LEFT JOIN age_segment AS a ON u.age_gr_id = a.age_gr_id
LEFT JOIN traffic_segment AS t ON u.tr_gr_id = t.tr_gr_id
LEFT JOIN lifetime_segment AS lt ON u.lt_gr_id = lt.lt_gr_id;
"""

In [12]:
main_table = pd.read_sql(query_1, engine)
main_table

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,nps_score,nps_group,country,city,age_segment,traffic_segment,lifetime_segment
0,A001A2,2320,Старый,45.0,женщина,ANDROID,SMARTPHONE,10,cторонники,Россия,Уфа,05 45-54,04 1-5,08 36+
1,A001WF,2344,Старый,53.0,мужчина,ANDROID,SMARTPHONE,10,cторонники,Россия,Киров,05 45-54,04 1-5,08 36+
2,A003Q7,467,Старый,57.0,мужчина,ANDROID,SMARTPHONE,10,cторонники,Россия,Москва,06 55-64,08 20-25,06 13-24
3,A004TB,4190,Старый,44.0,женщина,IOS,SMARTPHONE,10,cторонники,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+
4,A004XT,1163,Старый,24.0,мужчина,ANDROID,SMARTPHONE,10,cторонники,Россия,Рязань,02 16-24,05 5-10,08 36+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502488,ZZZKLD,1249,Старый,54.0,женщина,ANDROID,SMARTPHONE,5,критики,Россия,Москва,05 45-54,04 1-5,08 36+
502489,ZZZLWY,129,Новый,31.0,мужчина,ANDROID,SMARTPHONE,8,нейтралы,Россия,Москва,03 25-34,04 1-5,04 4-6
502490,ZZZQ5F,522,Старый,36.0,мужчина,ANDROID,SMARTPHONE,10,cторонники,Россия,Сургут,04 35-44,09 25-30,06 13-24
502491,ZZZQ8E,2936,Старый,37.0,женщина,ANDROID,SMARTPHONE,9,cторонники,Россия,УланУдэ,04 35-44,17 65-70,08 36+


In [13]:
main_table['gender_segment'].unique()

array(['женщина', 'мужчина', 'None'], dtype=object)

In [14]:
main_table.to_csv('telecomm.csv', index=False)

### Tableau Public board:

https://public.tableau.com/views/NSPproject/Dashboard1?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link

### Presentation:

https://disk.yandex.ru/d/H00HgvGUWyGKBg